# Erste Schritte mit RDDs

## Initialisierung von Spark

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (
        SparkSession
         .builder
         .appName("test")
         .master("local[4]")
         .getOrCreate()
        )

23/08/21 09:39:12 WARN Utils: Your hostname, pupil-a resolves to a loopback address: 127.0.1.1; using 167.235.141.210 instead (on interface eth0)
23/08/21 09:39:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/21 09:39:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


spark

In [4]:
sc = spark.sparkContext

## Erzeuge deinen ersten RDD

* [PySpark Docu](https://spark.apache.org/docs/latest/api/python/getting_started/index.html)
* [RDD Python API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.html#rdd-apis)

In [25]:
number_rdd = sc.parallelize([1, 2, 3, 4])

In [26]:
type(number_rdd)

pyspark.rdd.RDD

In [27]:
# wieviele Partitionen wurden erzeugt
number_rdd.getNumPartitions()

4

In [24]:
# alles ausgeben
number_rdd.collect()

[1, 2, 3, 4]

In [13]:
# nur die erste Zeile ausgeben
number_rdd.first()

1

In [14]:
# die ersten zwei Zeilen ausgeben
number_rdd.take(2)

[1, 2]

In [16]:
# auf gerade Zahlen filtern
even_number_rdd = number_rdd.filter(lambda x : x % 2 == 0)
even_number_rdd.collect()

[2, 4]

In [17]:
# ein leicht komplexeres Beispiel
address_rdd = sc.parallelize(
    [
        [1, "Marc", "Dortmund"],
        [2, "Bernd", "Hamburg"]
    ]
)

In [18]:
address_rdd.first()

[1, 'Marc', 'Dortmund']

In [19]:
people_from_hamburg_rdd = address_rdd.filter(lambda x: x[2]=='Hamburg')

In [20]:
people_from_hamburg_rdd.collect()

[[2, 'Bernd', 'Hamburg']]